In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%matplotlib inline
!pip install -qqq greykite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.7/22.7 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.9/499.9 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 49.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.0 MB/s et

In [3]:
# !pip install -qqq pandas
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Measles/benin.csv')

df['Year-Month'] = df['Year-Month'].astype('datetime64[ns]')
df.rename(columns = {'Year-Month': 'ts', 'Cases': 'y'}, inplace = True)
df

,ts,y
0,2011-01-01,21
1,2011-02-01,36
2,2011-03-01,49
3,2011-04-01,71
4,2011-05-01,50
...,...,...
154,2023-11-01,15
155,2023-12-01,28
156,2024-01-01,29
157,2024-02-01,83


In [4]:
from collections import defaultdict
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import plotly

import pandas as pd
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.autogen.forecast_config import EvaluationPeriodParam
from greykite.framework.templates.autogen.forecast_config import EvaluationMetricParam



from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

In [5]:
# Specifies dataset information
metadata = MetadataParam(
     time_col="ts",  # name of the time column
     value_col="y",  # name of the value column
     freq="MS"  #"MS" for Montly at start date, "H" for hourly, "D" for daily, "W" for weekly, etc.
 )

model_components = ModelComponentsParam(
    seasonality=None,
    growth={
        "growth_term": "linear"   #linear, quadratic, sqrt, None
    },
    events={
        "holidays_to_model_separately": None  # "auto",["New Year's Day", "Thanksgiving"],None]
    },
    changepoints={
        "changepoints_dict": {
            "method": "auto",  # 'auto' to automatically detect changepoints
            "regularization_strength": 0.5,  # Regularization strength to control number of changepoints
            "potential_changepoint_n": 5,  # Number of potential changepoints to consider
            "no_changepoint_proportion_from_end": 0.1  # Proportion of data at the end where no changepoint is allowed
        }
    },

    uncertainty={
        "uncertainty_dict":  "auto"  #`dict` or "auto" or None

    },
    custom={
        "fit_algorithm_dict": {
            "fit_algorithm": "quantile_regression"    #ridge, lasso, elastic_net, linear, quantile_regression
        }
    }

)

In [ ]:
# Specifies dataset information
metadata = MetadataParam(
     time_col="ts",  # name of the time column
     value_col="y",  # name of the value column
     freq="MS"  #"MS" for Montly at start date, "H" for hourly, "D" for daily, "W" for weekly, etc.
 )

model_components = ModelComponentsParam(
    seasonality={
        "yearly_seasonality": False,
        "quarterly_seasonality":False,
        "monthly_seasonality":False,
        "weekly_seasonality": False,
        "daily_seasonality": False,
        "auto_seasonality":False
    },
    growth={
        "growth_term": None     #linear, quadratic, sqrt, None
    },
    events={
        "holidays_to_model_separately": "auto"
    },
    custom={
        "fit_algorithm_dict": {
            "fit_algorithm": "ridge"
        }
    },
   changepoints={
        "changepoints_dict": {
            "method": "auto",  # 'auto' to automatically detect changepoints
            "regularization_strength": 0.5,  # Regularization strength to control number of changepoints
            "potential_changepoint_n": 25,  # Number of potential changepoints to consider
            "no_changepoint_proportion_from_end": 0.1  # Proportion of data at the end where no changepoint is allowed
        }
    },
    changepoints={
        "changepoints_dict": None         #"dict" or None
    },
      regressors={
        "growth_term": None
    },
    lagged_regressors={
        "growth_term": None
    },
    autoregression={
        "growth_term": None
    },
    uncertainty={
        "uncertainty_dict": None  #`dict` or "auto" or None

    },
      uncertainty={
        "uncertainty_dict": {
            "uncertainty_method": "simple_conditional_residuals",
            "params": {
                "conditional_cols": None,
                "quantile_estimation_method": "normal_fit",
                "sample_size_thresh": 5,
                "small_sample_size_method": "std_quantiles",
                "small_sample_size_quantile": 0.98
            }
        }
    },
      hyperparameter_override={
        "growth_term": None
    },
    custom={
        "fit_algorithm_dict": {
            "fit_algorithm": "quantile_regression"    #ridge, lasso, elastic_net, linear, quantile_regression
        }
    }
)

SyntaxError: keyword argument repeated: custom (<ipython-input-9-9d3362255604>, line 67)

In [6]:
# Specifies dataset information
metadata = MetadataParam(
     time_col="ts",  # name of the time column
     value_col="y",  # name of the value column
     freq="MS"  #"MS" for Montly at start date, "H" for hourly, "D" for daily, "W" for weekly, etc.
 )

forecaster = Forecaster()
result = forecaster.run_forecast_config(
     df=df,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE.name,
         forecast_horizon=12,  # forecasts 100 steps ahead
         coverage=0.95,  # 95% prediction intervals
         metadata_param=metadata,
         model_components_param=model_components

    )
)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


In [7]:
ts = result.timeseries
fig = ts.plot()
plotly.io.show(fig)

In [8]:
backtest = result.backtest
fig = backtest.plot()
plotly.io.show(fig)

In [9]:
forecast_testt = result.backtest.df_test.forecast
actual_testt = result.backtest.df_test.actual

In [10]:
forecast_testt

147    21.333979
148    20.269027
149    19.988838
150    21.890351
151    20.710226
152    21.076833
153    19.939119
154    18.988406
155    16.629862
156    26.779668
157    22.885216
158    23.227234
Name: forecast, dtype: float64

In [11]:
actual_testt

147    63
148    48
149    26
150    15
151    20
152    11
153    16
154    15
155    28
156    29
157    83
158    24
Name: actual, dtype: int64

In [21]:
from sklearn.metrics import mean_squared_error
error = np.sqrt(mean_squared_error(actual_testt,forecast_testt))
#print('Test RMSE: %.4f' % error)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(actual_testt,forecast_testt)
#print("Mean Absolute Error (MAE):", mae)

from sklearn.metrics import mean_squared_log_error
msle = mean_squared_log_error(actual_testt,forecast_testt)
#print("Mean Squared Logarithmic Error (MSLE):", msle)

def mean_absolute_percentage_error(y_true, y_pred):

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100



print('RMSE: %.2f' % error)
mape = mean_absolute_percentage_error(actual_testt, forecast_testt)
print(f"MAPE: {mape:.2f}%")
print('MAE:  %.2f' % mae)
print('MSLE: %.2f' % msle)

RMSE: 23.22
MAPE: 38.60%
MAE:  14.62
MSLE: 0.36


In [13]:
backtest_eval = defaultdict(list)
for metric, value in backtest.train_evaluation.items():
    backtest_eval[metric].append(value)
    backtest_eval[metric].append(backtest.test_evaluation[metric])
metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
metrics

,train,test
CORR,0.203615,0.201291
R2,-0.10099,-0.207453
MSE,2155.515648,539.228385
RMSE,46.427531,23.221292
MAE,23.572626,14.624259
MedAE,8.065726,6.450757
MAPE,NaN,38.603648
MedAPE,NaN,33.598506
sMAPE,43.75366,22.124686
Q80,16.691291,10.419161


In [15]:
# Access forecasted values
forecast_values = result.forecast.df

# Access forecasted components (e.g., trend, seasonality)
forecast_components = result.backtest.df_test
forecast_asd = result.forecast.df_test

# Print or display these results as needed
print("Forecast Values:")
print(forecast_components)


Forecast Values:
            ts  actual   forecast  forecast_lower  forecast_upper
147 2023-04-01      63  21.333979      -64.124236      106.792194
148 2023-05-01      48  20.269027      -65.189188      105.727241
149 2023-06-01      26  19.988838      -65.469377      105.447053
150 2023-07-01      15  21.890351      -63.567864      107.348566
151 2023-08-01      20  20.710226      -64.747989      106.168441
152 2023-09-01      11  21.076833      -64.381382      106.535048
153 2023-10-01      16  19.939119      -65.519096      105.397334
154 2023-11-01      15  18.988406      -66.469809      104.446621
155 2023-12-01      28  16.629862      -68.828353      102.088077
156 2024-01-01      29  26.779668      -58.678547      112.237883
157 2024-02-01      83  22.885216      -62.572999      108.343430
158 2024-03-01      24  23.227234      -62.230981      108.685449


In [16]:
backtest_eval = defaultdict(list)
for metric, value in backtest.train_evaluation.items():
     backtest_eval[metric].append(value)
     backtest_eval[metric].append(backtest.test_evaluation[metric])
metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
metrics

,train,test
CORR,0.203615,0.201291
R2,-0.10099,-0.207453
MSE,2155.515648,539.228385
RMSE,46.427531,23.221292
MAE,23.572626,14.624259
MedAE,8.065726,6.450757
MAPE,NaN,38.603648
MedAPE,NaN,33.598506
sMAPE,43.75366,22.124686
Q80,16.691291,10.419161
